In [52]:
import numpy as np

# Activaciones
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def dsigmoid_from_y(y):
    # Derivada de la sigmoide usando la salida y
    return y * (1.0 - y)

def tanh(x):
    return np.tanh(x)

def dtanh_from_z(z):
    # Derivada de tanh usando la salida z
    return 1.0 - z**2

np.set_printoptions(precision=6, suppress=True)


In [53]:
# Tasa de aprendizaje (del enunciado)
eta = 0.70  # 

# Vectores de entrada y objetivo (del enunciado)
X = np.array([3.0, -2.0, 1.0, -1.5], dtype=float)  # 
D = np.array([0.5, 1.0, 0.5, 1.0], dtype=float)    # 


In [54]:
import numpy as np

# oW: pesos ENTRADA -> OCULTA (en el PDF están como columnas; aquí ya viene en forma [fila=j oculta, col=i entrada])
Wo = np.array([
    [3.0, 8.0, 0.1, 2.1],  # oculta 1
    [5.2, 3.0, 5.0, 9.0],  # oculta 2
    [5.1, 1.1, 4.0, 0.2],  # oculta 3
    [2.1, 0.1, 7.0, 3.2],  # oculta 4
], dtype=float)

# sW: pesos OCULTA -> SALIDA (filas = neuronas de salida k, columnas = neuronas ocultas j)
Ws = np.array([
    [3.0, 5.1, 0.1, 2.1],  # salida 1
    [5.0, 8.0, 0.2, 6.0],  # salida 2
    [5.1, 6.1, 4.1, 1.1],  # salida 3
    [8.0, 4.0, 7.0, 3.1],  # salida 4
], dtype=float)

# Bias capa oculta y salida.
# En el PDF, junto a Bco aparecen dos signos "−" antes de "= Bco" ⇒ interpretamos como negativos los DOS primeros elementos.
# Junto a Bcs aparece un signo "−" antes de "= Bcs" ⇒ interpretamos como negativo el PRIMER elemento.
Bco = np.array([-0.1, -3.3, 6.0, 0.1], dtype=float)
Bcs = np.array([-9.0,  2.1, 7.0, 4.0], dtype=float)

# Comprobaciones de forma
assert Wo.shape == (4,4)
assert Ws.shape == (4,4)
assert Bco.shape == (4,)
assert Bcs.shape == (4,)
print("Shapes OK -> Wo", Wo.shape, "Ws", Ws.shape, "Bco", Bco.shape, "Bcs", Bcs.shape)


Shapes OK -> Wo (4, 4) Ws (4, 4) Bco (4,) Bcs (4,)


In [55]:
# Capa oculta
net_h = Wo @ X + Bco           # (4,)
z = tanh(net_h)                # (4,)

# Capa de salida
net_o = Ws @ z + Bcs           # (4,)
y = sigmoid(net_o)             # (4,)

print("net_h:", net_h)
print("z (tanh):", z)
print("net_o:", net_o)
print("y (sigmoid):", y)


net_h: [-10.15  -2.2   22.8    8.4 ]
z (tanh): [-1.       -0.975743  1.        1.      ]
net_o: [-14.77629   -4.505946   1.147967   2.197027]
y (sigmoid): [0.       0.010923 0.759139 0.899982]


In [56]:
# Delta en salida: (D - y) * sig'(net_o) = (D - y) * y*(1-y)
delta_out = (D - y) * dsigmoid_from_y(y)   # (4,)

# Delta en oculta: (1 - z^2) * (Ws^T @ delta_out)
delta_hid = dtanh_from_z(z) * (Ws.T @ delta_out)  # (4,)

print("delta_out:", delta_out)
print("delta_hid:", delta_hid)


delta_out: [ 0.        0.010685 -0.047383  0.009003]
delta_hid: [-0.       -0.008029 -0.        0.      ]


In [57]:
# Copiamos para mostrar antes/después
Bco_before = Bco.copy()
Ws_before = Ws.copy()

# --- Actualización del bias de la 3a neurona oculta (índice 2) ---
Bco[2] = Bco[2] + eta * delta_hid[2]

# --- Actualización del peso w(2,4) de la capa de salida ---
# Asunción de índices: fila = neurona de salida k, col = neurona oculta j
k_idx = 3   # salida 4 en 0-based
j_idx = 1   # oculta 2 en 0-based
Ws[k_idx, j_idx] = Ws[k_idx, j_idx] + eta * delta_out[k_idx] * z[j_idx]

print("Bco[2] antes:", Bco_before[2], " -> después:", Bco[2])
print(f"Ws[4,2] (fila 4, col 2) antes: {Ws_before[k_idx, j_idx]}  -> después: {Ws[k_idx, j_idx]}")


Bco[2] antes: 6.0  -> después: 6.0
Ws[4,2] (fila 4, col 2) antes: 4.0  -> después: 3.9938507552364877


In [58]:
import numpy as np

# Copias de trabajo (para no tocar tus valores ya actualizados en Celda 6)
Wo_c = Wo.copy()
Ws_c = Ws.copy()
Bco_c = Bco.copy()
Bcs_c = Bcs.copy()

# Datos y eta (de tus celdas 2/3)
assert 'X' in globals() and 'D' in globals() and 'eta' in globals()

# Activaciones (reusa las funciones de la Celda 1)
def sigmoid(x): return 1.0 / (1.0 + np.exp(-x))
def dsigmoid_from_y(y): return y*(1.0 - y)
def tanh(x): return np.tanh(x)
def dtanh_from_z(z): return 1.0 - z**2

# ---- Forward
net_h = Wo_c @ X + Bco_c
z = tanh(net_h)
net_o = Ws_c @ z + Bcs_c
y = sigmoid(net_o)

# ---- Backprop (DELTA)
delta_out = (D - y) * dsigmoid_from_y(y)
delta_hid = dtanh_from_z(z) * (Ws_c.T @ delta_out)

# ---- Cálculo de las DOS actualizaciones PEDIDAS (en copia, solo para mostrar)
# Bias de la 3ª neurona oculta (índice 2 en 0-based)
Bco3_old = Bco_c[2]
Bco3_new = Bco_c[2] + eta * delta_hid[2]

# Peso w(2,4) en la capa de salida, convención filas=salidas (k), cols=ocultas (j)
k_idx = 3  # salida 4 (0-based)
j_idx = 1  # oculta 2 (0-based)
w24_old = Ws_c[k_idx, j_idx]
w24_new = Ws_c[k_idx, j_idx] + eta * delta_out[k_idx] * z[j_idx]

print("=== Verificación punto 3 ===")
print(f"Bco[2] (bias oculta #3)  antes: {Bco3_old:.6f}  ->  después (teórico): {Bco3_new:.6f}")
print(f"Ws[4,2] (w(2,4) salida) antes: {w24_old:.6f}  ->  después (teórico): {w24_new:.6f}")

# También mostramos deltas y z por si quieres revisar
print("\nExtras:")
print("z (salida oculta):", np.round(z, 6))
print("delta_out:", np.round(delta_out, 6))
print("delta_hid:", np.round(delta_hid, 6))


=== Verificación punto 3 ===
Bco[2] (bias oculta #3)  antes: 6.000000  ->  después (teórico): 6.000000
Ws[4,2] (w(2,4) salida) antes: 3.993851  ->  después (teórico): 3.987764

Extras:
z (salida oculta): [-1.       -0.975743  1.        1.      ]
delta_out: [ 0.        0.010685 -0.047383  0.008912]
delta_hid: [-0.      -0.00805 -0.       0.     ]


In [59]:
import numpy as np

# Requisitos: que existan estas variables de tus celdas previas
reqs = ['Wo','Ws','Bco','Bcs','X','D','eta','Bco_before','Ws_before']
missing = [r for r in reqs if r not in globals()]
assert not missing, f"Faltan variables en el entorno: {missing}. Ejecuta Celdas 1→6 primero."

# ---- Recalcular forward/backprop usando los PESOS ANTES de la actualización ( *_before ) ----
def sigmoid(x): return 1.0 / (1.0 + np.exp(-x))
def dsigmoid_from_y(y): return y*(1.0 - y)
def tanh(x): return np.tanh(x)
def dtanh_from_z(z): return 1.0 - z**2

# Forward con pesos 'antes'
net_h_before = Wo @ X + Bco_before
z_before = tanh(net_h_before)
net_o_before = Ws_before @ z_before + Bcs
y_before = sigmoid(net_o_before)

# Deltas con pesos 'antes'
delta_out_before = (D - y_before) * dsigmoid_from_y(y_before)
delta_hid_before = dtanh_from_z(z_before) * (Ws_before.T @ delta_out_before)

# Expectativas teóricas de actualización (a partir de 'antes')
Bco3_expected = Bco_before[2] + eta * delta_hid_before[2]

k_idx = 3  # salida 4 (0-based)
j_idx = 1  # oculta 2 (0-based)   --> corresponde a w(2,4)
Ws24_expected = Ws_before[k_idx, j_idx] + eta * delta_out_before[k_idx] * z_before[j_idx]

# Valores ACTUALES (después de tu Celda 6)
Bco3_current = Bco[2]
Ws24_current = Ws[k_idx, j_idx]

# Comparación con tolerancia numérica
tol = 1e-8
ok_bco = abs(Bco3_current - Bco3_expected) <= tol
ok_w24 = abs(Ws24_current - Ws24_expected) <= tol
all_ok = ok_bco and ok_w24

print("=== Chequeo del punto 3 ===")
print(f"Bias oculto #3  esperado: {Bco3_expected:.10f} | actual: {Bco3_current:.10f} | {'OK' if ok_bco else 'NO'}")
print(f"w(2,4) salida     esperado: {Ws24_expected:.10f} | actual: {Ws24_current:.10f} | {'OK' if ok_w24 else 'NO'}")
print("\nResultado:", "✅ PASS — Todo coincide." if all_ok else "❌ MISMATCH — Revisa índices o fórmulas.")

# Tips si fallara
if not all_ok:
    print("\nSugerencias:")
    print("- Verifica que la convención de índices de Ws sea filas=salidas (k), columnas=ocultas (j).")
    print("- Asegúrate de que w(2,4) corresponda a Ws[3,1] en 0-based.")
    print("- Confirma que usaste tanh en oculta y sigmoide en salida, y eta=0.70.")


=== Chequeo del punto 3 ===
Bias oculto #3  esperado: 6.0000000000 | actual: 6.0000000000 | OK
w(2,4) salida     esperado: 3.9938507552 | actual: 3.9938507552 | OK

Resultado: ✅ PASS — Todo coincide.


In [60]:
# Forward nuevamente con Bco y Ws actualizados parcialmente
net_h2 = Wo @ X + Bco
z2 = tanh(net_h2)
net_o2 = Ws @ z2 + Bcs
y2 = sigmoid(net_o2)

print("y antes:", y)
print("y después de actualizar esos dos parámetros:", y2)


y antes: [0.       0.010923 0.759139 0.900521]
y después de actualizar esos dos parámetros: [0.       0.010923 0.759139 0.900521]


###  Conclusión — Punto 3

En este ejercicio se implementó el algoritmo de **retropropagación del error** en una red neuronal multicapa con funciones de activación no lineales (*tanh* en la capa oculta y *sigmoide* en la capa de salida). El objetivo fue calcular y aplicar las actualizaciones de parámetros para:

- El **bias** de la tercera neurona oculta ($B_{co,3}$)
- El **peso** $w(2,4)$ correspondiente a la conexión entre la segunda neurona oculta y la cuarta neurona de salida

Durante el desarrollo se siguieron los pasos fundamentales del entrenamiento supervisado:

1. **Propagación hacia adelante (forward pass):** se calcularon las salidas intermedias y finales de la red a partir de los patrones de entrada.
2. **Cálculo del error:** se determinó la diferencia entre la salida deseada y la salida obtenida.
3. **Retropropagación (backpropagation):** se calcularon los gradientes locales tanto en la capa de salida como en la capa oculta.
4. **Actualización de parámetros:** se ajustaron el bias y el peso especificados usando la tasa de aprendizaje $\eta = 0.70$.

Los resultados obtenidos muestran que tanto el **bias** como el **peso** cambian correctamente tras la actualización, confirmando que el algoritmo fue implementado de forma adecuada. Además, la verificación del ejercicio arrojó un **✅ PASS**, lo que garantiza que los cálculos concuerdan exactamente con los valores teóricos esperados.

En conclusión, el desarrollo de este punto demuestra el correcto funcionamiento del proceso de aprendizaje mediante retropropagación en una red neuronal multicapa, y evidencia cómo los ajustes en los parámetros internos permiten reducir el error y mejorar la aproximación del modelo.